In [12]:
import sys
from pathlib import Path

# Add the backend directory to the Python path
backend_dir = Path.cwd().parent
sys.path.insert(0, str(backend_dir))

In [6]:
from openai import AsyncOpenAI
import os
from pathlib import Path

client = AsyncOpenAI(
    base_url=os.environ.get('LITELLM_BASE_URL'),
    api_key=os.environ.get('LITELLM_API_KEY'),
)

In [7]:
upload = await client.files.create(
    file=Path('./files/sample1.pdf'),
    purpose='user_data',
)
upload

FileObject(id='file-CqHBSiKG8GPEgB9dMmDK1T', bytes=50919, created_at=1755433787, filename='sample1.pdf', object='file', purpose='user_data', status='processed', expires_at=None, status_details=None)

In [23]:
from src.schemas.llm.education import EducationLLMSchema

output = await client.responses.parse(
    model='gpt-5-mini',
    text_format=EducationLLMSchema,
    reasoning={
        'effort': 'medium',
    },
    input=[
        {
            'role': 'user',
            'content': [
                {
                    'type': 'input_text',
                    'text': 'Help extract the education experience of this user.'
                },
                {
                    'type': 'input_file',
                    'file_id': upload.id
                }
            ]
        }
    ]
)

In [26]:
output_items = output.output
for item in output_items:
    if item.type == 'message':
        content_items = item.content
        for content in content_items:
            if content.type == 'output_text':
                print(content.parsed.model_dump_json(indent=2))

{
  "institution_name": "National University of Singapore",
  "degree": "bachelor",
  "field_of_study": "Computer Science",
  "focus_area": "Selected for NUS Overseas College (NOC) Stockholm — year-long exchange and overseas internship; participated in classes at KTH Royal Institute of Technology and Stockholm School of Economics",
  "start_date": "August 2021",
  "end_date": "December 2025 (Expected)",
  "gpa": 4.71,
  "max_gpa": 5.0
}
